<a href="https://colab.research.google.com/github/sudo-ashu/project-pharma/blob/main/projectPharma.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import kagglehub
pharma_data_path = kagglehub.dataset_download('milanzdravkovic/pharma-sales-data')

print('data import complete')

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import os
import statsmodels.api as sm
import statsmodels
#import datetime for dates and time realted calculations
import datetime as dt
from pandas.plotting import register_matplotlib_converters
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.stattools import acf, pacf
from statsmodels.tsa.arima_model import ARIMA
register_matplotlib_converters()
from time import time

# **Data Reading**

In [ ]:
#reading the data
# (not needed data) hourly = pd.read_csv('/kaggle/input/pharma-sales-data/saleshourly.csv')
daily = pd.read_csv('/kaggle/input/pharma-sales-data/salesdaily.csv')
weekly = pd.read_csv('/kaggle/input/pharma-sales-data/salesweekly.csv')
monthly = pd.read_csv('/kaggle/input/pharma-sales-data/salesmonthly.csv')

# **Analysing Data**

In [ ]:
#function to print shape of a given data
def print_shape(data):
    print('Rows : ',data.shape[0])
    print('Columns : ',data.shape[1])

In [ ]:
# print('Hourly Data')
# print_shape(hourly)
print('Daily Data')
print_shape(daily)
print('Weekly Data')
print_shape(weekly)
print('Monthly Data')
print_shape(monthly)

In [ ]:
daily.describe()

In [ ]:
weekly.describe()

In [ ]:
monthly.describe()

we see that the datum colum (which contanis the date) is of type object. So, we have to change its type to datetime type in each csv file. But before that we will make copies of the csv files.

In [ ]:
#copy the data
# (not needed) copy_hourly = hourly.copy()
copy_daily = daily.copy()
copy_weekly = weekly.copy()
copy_monthly = monthly.copy()

In [ ]:
# hourly['datum'] = pd.to_datetime(hourly['datum'], format= '%m/%d/%Y %H:%M')
daily['datum'] = pd.to_datetime(daily['datum'], format= '%m/%d/%Y')
weekly['datum'] = pd.to_datetime(weekly['datum'], format= '%m/%d/%Y')
monthly['datum'] = pd.to_datetime(monthly['datum'], format= '%Y-%m-%d')

# **seeing the monthly series**

In [ ]:
# extracting year, month and day from datum
monthly['year'] = monthly['datum'].dt.year
monthly['month'] = monthly['datum'].dt.month
monthly['day'] = monthly['datum'].dt.day

In [ ]:
# replacing index of the table from date for easy analysis
monthly.set_index(monthly['datum'], inplace=True)

In [ ]:
monthly.isnull().sum()

this implies therea are zero null values

In [ ]:
monthly.sample(3)

In [ ]:
monthly.tail(3)

In [ ]:
monthly.columns

In [ ]:
med_columns = ['M01AB', 'M01AE', 'N02BA', 'N02BE', 'N05B', 'N05C', 'R03', 'R06']
for col in med_columns:
    plt.figure(figsize=(10, 6))
    plt.plot(monthly.index, monthly[col], 'ro-')
    plt.title(f'{col} Time Series')
    plt.xlabel(col)
    plt.ylabel('Sales Value')
    plt.show()

Adding total-sales column in monthly table

In [ ]:
monthly['total_sales'] = monthly['M01AB']
for cols in monthly.columns[2:9]:
    monthly['total_sales'] = monthly['total_sales'] + monthly[cols]

In [ ]:
# plotting the total sales data
plt.figure(figsize=(10, 6))
plt.plot(monthly.index, monthly['total_sales'], 'ro-')
plt.xlabel('Date Time')
plt.ylabel('Total Sales')
plt.title('Total Sales of Drugs')
plt.show()

In [ ]:
# calculating the Simple Moving Average with a week size of 4
window_size = 4
monthly['SMA'] = monthly['total_sales'].rolling(window_size, min_periods=1).mean()

# Now plotting this SMA and total Sales on a single graph
plt.figure(figsize=(10, 6))
plt.plot(monthly.index, monthly['total_sales'], label='Original Data')
plt.plot(monthly.index, monthly['SMA'], 'ro-', label=f'SMA ({window_size}-week window)')
plt.title('Monthly Drug Sales with Simple Moving Average')
plt.xlabel('Date')
plt.ylabel('Drug Sales')
plt.legend()
plt.show()

As SMA smoothens out noise from the time series. We can clearly see the red coloured curve has lower fluctuations. Also the sales of drugs is consistent over the years. Also, the seasonality at the end of each year is very clear as well. The sales at the end of each year increases.

In [ ]:
# cheching for stationarity of the time series
def perform_adf_test(series):
    result = adfuller(series)
    print('ADF Statistic: %f' % result[0])
    print('p-value: %f' % result[1])

In [ ]:
perform_adf_test(monthly["total_sales"])

In [ ]:
perform_adf_test(monthly["SMA"])

Chechking ACF and PACF plots

In [ ]:
# infer the frequency of the data
monthly = monthly.asfreq(pd.infer_freq(monthly.index))

In [ ]:
# Plot ACF and PACF of the differenced series
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(10, 8))
sm.graphics.tsa.plot_acf(monthly['total_sales'], lags=30, ax=ax1)
sm.graphics.tsa.plot_pacf(monthly['total_sales'], lags=30, ax=ax2)
plt.show()

p, q should be 1, 1 according to ACF, PACF tests respectively

# ARMA(1,1) model or simply using AR(1) or MA(1) model to fit our time series

# Train test split to check the efficiency of our model

In [ ]:
train_end = dt.datetime(2019,7,31)
# test_end = datetime(2004,1,1) till end

train_data = monthly[:train_end]
test_data = monthly[train_end + dt.timedelta(days=1):]

We will forecast only 2 months of data in advance because prediction beyond a certain limit is useless and leads to error

## Not using ARMA model because the p values for coefficients of MA model are more than 0.05, that's why rejecting those values and sticking to AR model

In [ ]:
# Fit an ARIMA model to the  total sales series
# p, d, q = 1, 0, 1  # You can adjust these values based on ACF and PACF analysis

model = statsmodels.tsa.ar_model.AutoReg(train_data["total_sales"], lags=1)
results = model.fit()

In [ ]:
# Print model summary
print(results.summary())

Means AR model'S coefficient is useful but the MA seems to be useless as per its p value

## So our series follow AR(1) model which is:
$\hat{y_t} = 1182.0998 + 0.3558y_{t-1}  $

In [ ]:
test_data

In [ ]:
# Forecast the next 10 time steps
forecast = results.forecast(steps=3)

# Plot the forecast
plt.figure(figsize=(10, 6))
plt.plot(train_data.index, train_data['total_sales'], label='Observed')
plt.plot(test_data.index, test_data['total_sales'], label='True Test Data')
plt.plot(forecast.index, forecast, label='Forecast')
plt.title('Total Sales of Drugs Forecast')
plt.xlabel('Date')
plt.ylabel('Total Sales')
plt.legend()
plt.show()

In [ ]:
forecast

In [ ]:
residuals = test_data["total_sales"] - forecast

In [ ]:
plt.figure(figsize=(10,4))
plt.plot(residuals, 'bo-')
plt.title('Residuals from AR Model', fontsize=20)
plt.ylabel('Error', fontsize=16)
plt.axhline(0, color='g', linestyle='-', alpha=0.2)

In [ ]:
print('Root Mean Squared Error:', np.sqrt(np.mean(residuals**2)))

# Findings
The AR(1) model works fine for predicting just 2 data points but root mean square error is high due to last data point.